# Three-grid P-multigrid Example

In [ ]:
# dependencies
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.develop(path="../..")
using Plots
using LFAToolkit
using LinearAlgebra

In [ ]:
# parameters
finep            = 4;
midp             = 2;
coarsep          = 1;
numbercomponents = 1;
dimension        = 1;
numbersteps      = 100;
θ_min            = -π;
p                = [2];
v                = [1, 1];

In [ ]:
# setup
mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

mtofbasis = TensorH1LagrangePProlongationBasis(midp + 1, finep + 1, numbercomponents, dimension)
ctombasis = TensorH1LagrangePProlongationBasis(coarsep + 1, midp + 1, numbercomponents, dimension)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep + 1, finep + 1, mesh)
middiffusion = GalleryOperator("diffusion", midp + 1, finep + 1, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep + 1, finep + 1, mesh)

# Chebyshev smoother
finechebyshev = Chebyshev(finediffusion)
midchebyshev = Chebyshev(middiffusion)

# p-multigrid preconditioner
midmultigrid = PMultigrid(middiffusion, coarsediffusion, midchebyshev, [ctombasis])
multigrid = PMultigrid(finediffusion, midmultigrid, finechebyshev, [mtofbasis])

In [ ]:
# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # compute
    (θ_range, eigenvalues, _) = computesymbolsoverrange(multigrid, p, v, numbersteps; θ_min = θ_min)
    maxeigenvalues = maximum(real(eigenvalues); dims = 2)
    mineigenvalues = minimum(real(eigenvalues); dims = 2)

    # plot
    println("max eigenvalue: ", maximum(maxeigenvalues))
    xrange = θ_range/π
    plot(
        xrange,
        xlabel = "θ/π",
        xtickfont = font(12, "Courier"),
        [maxeigenvalues, mineigenvalues],
        ytickfont = font(12, "Courier"),
        ylabel = "spectral radius",
        linewidth = 3,
        legend = :none,
        title = "P-Multigrid Symbol Maximal Eigenvalues"
    )
    ymin = minimum(mineigenvalues)
    ylims!(minimum([0, ymin * 1.1]), maximum(maxeigenvalues) * 1.1)
# -- 2D --
elseif dimension == 2
    # compute
    (_, eigenvalues, _) = computesymbolsoverrange(multigrid, p, v, numbersteps; θ_min = θ_min)
    maxeigenvalues = reshape(maximum(real(eigenvalues); dims=2), (numbersteps, numbersteps))
    mineigenvalues = reshape(minimum(real(eigenvalues); dims=2), (numbersteps, numbersteps))
    
    # plot
    θ_max = θ_min + 2π
    θ_range = LinRange(θ_min, θ_max, numbersteps)
    println("max eigenvalue: ", maximum(maxeigenvalues))
    xrange = θ_range/π
    plot1 = heatmap(
        xrange,
        xlabel = "θ/π",
        xtickfont = font(12, "Courier"),
        xrange,
        ylabel = "θ/π",
        maxeigenvalues,
        ytickfont = font(12, "Courier"),
        title = "P-Multigrid Symbol Maximum Eigenvalues",
        transpose = true,
        aspect_ratio = :equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
    plot2 = heatmap(
        xrange,
        xlabel = "θ/π",
        xtickfont = font(12, "Courier"),
        xrange,
        ylabel = "θ/π",
        mineigenvalues,
        ytickfont = font(12, "Courier"),
        title = "P-Multigrid Symbol Minimum Eigevalues",
        transpose = true,
        aspect_ratio = :equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
    plot!(plot1, plot2, layout = (2, 1), size = (700, 1400))
end